In [21]:
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
from scipy.optimize import curve_fit
from uncertainties import ufloat
from uncertainties import unumpy as unp
import array_to_latex as a2l
import csv
import pandas as pd
from astropy.io.votable import parse
from astropy.table import QTable, Table, Column
from functools import reduce

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']}) #font na grafih je LaTexov
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{siunitx}')

In [25]:
 def join(body_1, body_2):
    m_1, r_1, J_1 = body_1
    m_2, r_2, J_2 = body_2
    # New center of mass
    M = m_1 + m_2
    R = m_1/M * r_1 + m_2/M * r_2
    # Old centers of mass from new center of mass
    R_1 = r_1 - R
    R_2 = r_2 - R
    # Parallel axis theorem
    I = np.identity(3)
    J = (J_1 + J_2 +
        m_1 * (np.sum(R_1**2) * I - np.outer(R_1, R_1)) +
        m_2 * (np.sum(R_2**2) * I - np.outer(R_2, R_2))
    )
    return M, R, J

def J_sph(m, r):
    return 2/5 * m*r**2 * np.identity(3)

def J_cyl(m, r, h):
    J_xy = 1/4*m*r**2 + 1/12*m*h**2
    J_z = 1/2 * m*r**2
    return np.diag([J_xy, J_xy, J_z])

In [33]:
g = 9.802
# Sphere
m_s = unp.uarray([515e-3],[1e-3])
r_s = unp.uarray([50.8e-3/2], [0.1e-3/2])
# Ring
m_r = unp.uarray([15e-3], [1e-3])
r_r = unp.uarray([51e-3/2], [0.1e-3/2])
h_r = unp.uarray([1.1e-3], [0.1e-3/2])
# Bar
m_b = unp.uarray([27e-3], [1e-3])
r_b = unp.uarray([6.5e-3/2], [0.1e-3/2])
h_b = unp.uarray([100.5e-3], [0.1e-3])
# Weight
m_w = unp.uarray([18e-3], [1e-3])
r_w = unp.uarray([20e-3/2], [0.5e-3/2])
h_w = unp.uarray([25.2e-3], [0.1e-3])

def composite_body(l, weight=True):
    bodies = [
        [m_s, np.array([0, 0, 0], dtype=object), J_sph(m_s, r_s)], # Sphere
        [m_r, np.array([0, 0, r_s+h_r/2], dtype=object), J_cyl(m_r, r_r, h_r)], # Ring
        [m_b, np.array([0, 0, r_s+h_b/2], dtype=object), J_cyl(m_b, r_b, h_b)] # Bar
    ]
    if weight:
        bodies.append([m_w, np.array([0, 0, r_s+l+r_w/2], dtype=object), J_cyl(m_w, r_w, h_w)])
    return reduce(join, bodies)

In [48]:
l = 0
m, r_cm, J_cm = composite_body(l)
h_star = unp.sqrt(np.sum(r_cm**2))
J_11, J_22, J_33 = np.diag(J_cm)

v_z_rpm = unp.uarray([350, 510, 600], 3 * [50])

v_z_rpm = v_z_rpm / 60

v_pr = 1 / (2 * np.pi) ** 2 * m * g * h_star / (J_33 * v_z_rpm)

v_nu = J_33 / J_11 * v_z_rpm

data = np.array([unp.nominal_values(v_z_rpm) *60, unp.std_devs(v_z_rpm) * 60, unp.nominal_values(v_pr), unp.std_devs(v_pr), unp.nominal_values(v_nu), unp.std_devs(v_nu)]).T

#data = np.squeeze(data, axis=1).T 

print(a2l.to_ltx(data, frmt='{:6.2f}', arraytype='array'))



\begin{array}
  350.00 &   50.00 &    0.79 &    0.12 &    2.86 &    0.41\\
  510.00 &   50.00 &    0.55 &    0.06 &    4.17 &    0.41\\
  600.00 &   50.00 &    0.46 &    0.04 &    4.90 &    0.41
\end{array}
None


In [52]:
l = 3.5e-2
m, r_cm, J_cm = composite_body(l)
h_star = unp.sqrt(np.sum(r_cm**2))
J_11, J_22, J_33 = np.diag(J_cm)

v_z_rpm = unp.uarray([370, 510, 580], 3 * [50])

v_z_rpm = v_z_rpm / 60

v_pr = 1 / (2 * np.pi) ** 2 * m * g * h_star / (J_33 * v_z_rpm)

v_nu = J_33 / J_11 * v_z_rpm

data = np.array([unp.nominal_values(v_z_rpm) *60, unp.std_devs(v_z_rpm) * 60, unp.nominal_values(v_pr), unp.std_devs(v_pr), unp.nominal_values(v_nu), unp.std_devs(v_nu)]).T

print(a2l.to_ltx(data, frmt='{:6.2f}', arraytype='array'))


\begin{array}
  370.00 &   50.00 &    0.91 &    0.13 &    2.60 &    0.35\\
  510.00 &   50.00 &    0.66 &    0.07 &    3.58 &    0.35\\
  580.00 &   50.00 &    0.58 &    0.05 &    4.07 &    0.36
\end{array}
None


In [53]:
l = 6.5e-2
m, r_cm, J_cm = composite_body(l)
h_star = unp.sqrt(np.sum(r_cm**2))
J_11, J_22, J_33 = np.diag(J_cm)

v_z_rpm = unp.uarray([300, 410, 510], 3 * [50])

v_z_rpm = v_z_rpm / 60

v_pr = 1 / (2 * np.pi) ** 2 * m * g * h_star / (J_33 * v_z_rpm)

v_nu = J_33 / J_11 * v_z_rpm

data = np.array([unp.nominal_values(v_z_rpm) *60, unp.std_devs(v_z_rpm) * 60, unp.nominal_values(v_pr), unp.std_devs(v_pr), unp.nominal_values(v_nu), unp.std_devs(v_nu)]).T

print(a2l.to_ltx(data, frmt='{:6.2f}', arraytype='array'))

\begin{array}
  300.00 &   50.00 &    1.29 &    0.22 &    1.74 &    0.29\\
  410.00 &   50.00 &    0.94 &    0.12 &    2.38 &    0.29\\
  510.00 &   50.00 &    0.76 &    0.08 &    2.96 &    0.30
\end{array}
None
